### Notebook for creating dataset that will be uploaded to aws rds (MySQL)
### The weather dataset was created using the noaa_weather.py program.
##### In the future the weather may be aggregated somehow, this method is good enough to get the neural network set up and create the process for feeding the weather data to the network.

In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import geocoder

In [2]:
dfmgrs = pd.read_csv('c_us.csv', index_col= False, encoding = 'unicode_escape')

c:\users\tjlog\appdata\local\programs\python\python37\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return func(*args, **kwargs)


In [3]:
dfmgrs.head()

,Entity ID,Acquisition Start Date,Acquisition End Date,Tile Number,Cloud Cover,Agency,Platform,Vendor,Vendor Tile ID,Orbit Number,...,Center Longitude dec,NW Corner Lat dec,NW Corner Long dec,NE Corner Lat dec,NE Corner Long dec,SE Corner Lat dec,SE Corner Long dec,SW Corner Lat dec,SW Corner Long dec,Display ID
0,L1C_T10TFL_A029325_20210201T190617,2021-02-01T19:06:17.464Z,2021-02-01T19:14:17.298Z,T10TFL,27.1647,ESA,SENTINEL-2A,VGS2,L1C_T10TFL_A029325_20210201T190617,13,...,-121.157036,41.545594,-121.801030,41.524345,-120.485462,40.536186,-120.522802,40.556712,-121.818852,L1C_T10TFL_A029325_20210201T190617
1,L1C_T12STH_A020416_20210201T182238,2021-02-01T18:22:38.527Z,2021-02-01T18:26:58.287Z,T12STH,99.9947,ESA,SENTINEL-2B,VGS1,L1C_T12STH_A020416_20210201T182238,84,...,-113.803879,38.797895,-114.454503,38.828440,-113.191347,37.839596,-113.161752,37.810108,-114.407915,L1C_T12STH_A020416_20210201T182238
2,L1C_T12SVF_A020416_20210201T182238,2021-02-01T18:22:38.527Z,2021-02-01T18:26:58.287Z,T12SVF,99.9978,ESA,SENTINEL-2B,VGS1,L1C_T12SVF_A020416_20210201T182238,84,...,-111.504373,37.041250,-112.124935,37.046532,-110.890241,36.056675,-110.891632,36.051579,-112.110684,L1C_T12SVF_A020416_20210201T182238
3,L1C_T12STA_A020416_20210201T182238,2021-02-01T18:22:38.527Z,2021-02-01T18:26:58.287Z,T12STA,98.2564,ESA,SENTINEL-2B,VGS1,L1C_T12STA_A020416_20210201T182238,84,...,-113.595134,32.496814,-114.192952,32.521047,-113.025220,31.531070,-113.003545,31.507745,-114.158821,L1C_T12STA_A020416_20210201T182238
4,L1C_T11SQA_A020416_20210201T182238,2021-02-01T18:22:38.527Z,2021-02-01T18:26:58.287Z,T11SQA,99.9998,ESA,SENTINEL-2B,VGS1,L1C_T11SQA_A020416_20210201T182238,84,...,-114.153856,37.025284,-114.752070,36.995511,-113.519414,36.007451,-113.563414,36.036176,-114.780526,L1C_T11SQA_A020416_20210201T182238


In [4]:
dfmgrs = dfmgrs[['Tile Number', 'Center Latitude dec', 'Center Longitude dec']]

In [5]:
dfmgrs

,Tile Number,Center Latitude dec,Center Longitude dec
0,T10TFL,41.042580,-121.157036
1,T12STH,38.320709,-113.803879
2,T12SVF,36.550607,-111.504373
3,T12STA,32.015518,-113.595134
4,T11SQA,36.517700,-114.153856
...,...,...,...
64567,T13TDH,42.857152,-105.552529
64568,T12SYF,36.517700,-108.153856
64569,T13SCA,36.540655,-106.621013
64570,T13TCH,42.844714,-106.775699


In [6]:
dfmgrs = dfmgrs.drop_duplicates()

In [7]:
dfmgrs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406 entries, 0 to 818
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Tile Number           406 non-null    object 
 1   Center Latitude dec   406 non-null    float64
 2   Center Longitude dec  406 non-null    float64
dtypes: float64(2), object(1)
memory usage: 12.7+ KB


In [8]:
len(dfmgrs['Tile Number'].unique())

406

In [9]:
dfmgrs = dfmgrs.rename(columns = {'Tile Number': 'MGRS', 'Center Latitude dec': 'Latitude', 'Center Longitude dec': 'Longitude' })

In [10]:
dfmgrs['MGRS'] = dfmgrs['MGRS'].str[1:]

In [11]:
dfmgrs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406 entries, 0 to 818
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   MGRS       406 non-null    object 
 1   Latitude   406 non-null    float64
 2   Longitude  406 non-null    float64
dtypes: float64(2), object(1)
memory usage: 12.7+ KB


In [12]:
dfmgrs.head()

,MGRS,Latitude,Longitude
0,10TFL,41.042580,-121.157036
1,12STH,38.320709,-113.803879
2,12SVF,36.550607,-111.504373
3,12STA,32.015518,-113.595134
4,11SQA,36.517700,-114.153856


In [13]:
#not really sure how to vectorize and not worth the time to figure out
for index, row in tqdm(dfmgrs.iterrows(), total =len(dfmgrs['MGRS'].unique())):
    loc = geocoder.osm([row['Latitude'],row['Longitude']], method='reverse')
    try:
        ccode = loc.json['country_code']
    except:
        ccode = None
    try:
        country = loc.json['country']
    except:
        country = None
    try:    
        state = loc.json['state']
    except:
        state = None
    dfmgrs.loc[index, 'Country_Code'] = ccode
    dfmgrs.loc[index, 'Country'] = country
    dfmgrs.loc[index, 'STATE'] = state

  0%|          | 0/406 [00:00<?, ?it/s]

In [14]:
dfmgrs.head()

,MGRS,Latitude,Longitude,Country_Code,Country,STATE
0,10TFL,41.042580,-121.157036,us,United States,California
1,12STH,38.320709,-113.803879,us,United States,Utah
2,12SVF,36.550607,-111.504373,us,United States,Arizona
3,12STA,32.015518,-113.595134,mx,México,Sonora
4,11SQA,36.517700,-114.153856,us,United States,Nevada


In [15]:
from sqlalchemy import create_engine
import os
USER = os.environ.get('DBUSERNAME')
PASSWORD = os.environ.get('DBPASSWORD')
ENDPOINT = os.environ.get('DBDEV')
SCHEMA = 'dev'
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{ENDPOINT}/{SCHEMA}")

In [15]:
table = 'mgrs'
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{ENDPOINT}/{SCHEMA}")
dfmgrs.to_sql(table, con = engine, if_exists = 'append')

In [16]:
df = pd.read_csv('l2a_index.csv')

c:\users\tjlog\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3524: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13107242 entries, 0 to 13107241
Data columns (total 14 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   GRANULE_ID              object 
 1   PRODUCT_ID              object 
 2   DATATAKE_IDENTIFIER     object 
 3   MGRS_TILE               object 
 4   SENSING_TIME            object 
 5   TOTAL_SIZE              int64  
 6   CLOUD_COVER             float64
 7   GEOMETRIC_QUALITY_FLAG  object 
 8   GENERATION_TIME         object 
 9   NORTH_LAT               float64
 10  SOUTH_LAT               float64
 11  WEST_LON                float64
 12  EAST_LON                float64
 13  BASE_URL                object 
dtypes: float64(5), int64(1), object(8)
memory usage: 1.4+ GB


In [18]:
del df['PRODUCT_ID']
del df['DATATAKE_IDENTIFIER']
del df['GEOMETRIC_QUALITY_FLAG']
del df['GENERATION_TIME']
del df['NORTH_LAT']
del df['SOUTH_LAT']
del df['WEST_LON']
del df['EAST_LON']

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13107242 entries, 0 to 13107241
Data columns (total 6 columns):
 #   Column        Dtype  
---  ------        -----  
 0   GRANULE_ID    object 
 1   MGRS_TILE     object 
 2   SENSING_TIME  object 
 3   TOTAL_SIZE    int64  
 4   CLOUD_COVER   float64
 5   BASE_URL      object 
dtypes: float64(1), int64(1), object(4)
memory usage: 600.0+ MB


In [20]:
df.head()

,GRANULE_ID,MGRS_TILE,SENSING_TIME,TOTAL_SIZE,CLOUD_COVER,BASE_URL
0,L2A_T29TPF_A029063_20210114T112450,29TPF,2021-01-14T11:29:57.770000Z,1176133281,11.188985,gs://gcp-public-data-sentinel-2/L2/tiles/29/T/...
1,L2A_T53DND_A025853_20220216T235459,53DND,2022-02-16T23:56:07.936000Z,915771677,0.090527,gs://gcp-public-data-sentinel-2/L2/tiles/53/D/...
2,L2A_T35LML_A023284_20191207T083209,35LML,2019-12-07T08:42:35.745000Z,876400648,65.698431,gs://gcp-public-data-sentinel-2/L2/tiles/35/L/...
3,L2A_T15WVU_A021889_20210515T181914,15WVU,2021-05-15T18:24:14.976000Z,942897613,27.193338,gs://gcp-public-data-sentinel-2/L2/tiles/15/W/...
4,L2A_T54UXV_A018704_20190121T013955,54UXV,2019-01-21T01:43:27.000000Z,879335795,98.612651,gs://gcp-public-data-sentinel-2/L2/tiles/54/U/...


In [21]:
df['SENSING_TIME'] = pd.to_datetime(df['SENSING_TIME'])

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13107242 entries, 0 to 13107241
Data columns (total 6 columns):
 #   Column        Dtype              
---  ------        -----              
 0   GRANULE_ID    object             
 1   MGRS_TILE     object             
 2   SENSING_TIME  datetime64[ns, UTC]
 3   TOTAL_SIZE    int64              
 4   CLOUD_COVER   float64            
 5   BASE_URL      object             
dtypes: datetime64[ns, UTC](1), float64(1), int64(1), object(3)
memory usage: 600.0+ MB


In [23]:
df = df[df['SENSING_TIME'] >= '2019-01-01']

In [24]:
mgrs = dfmgrs.MGRS.unique().tolist()

In [25]:
df['PROCESSED'] = 0
df['IN_PROGRESS'] = 0

In [26]:
df.head()

,GRANULE_ID,MGRS_TILE,SENSING_TIME,TOTAL_SIZE,CLOUD_COVER,BASE_URL,PROCESSED,IN_PROGRESS
0,L2A_T29TPF_A029063_20210114T112450,29TPF,2021-01-14 11:29:57.770000+00:00,1176133281,11.188985,gs://gcp-public-data-sentinel-2/L2/tiles/29/T/...,0,0
1,L2A_T53DND_A025853_20220216T235459,53DND,2022-02-16 23:56:07.936000+00:00,915771677,0.090527,gs://gcp-public-data-sentinel-2/L2/tiles/53/D/...,0,0
2,L2A_T35LML_A023284_20191207T083209,35LML,2019-12-07 08:42:35.745000+00:00,876400648,65.698431,gs://gcp-public-data-sentinel-2/L2/tiles/35/L/...,0,0
3,L2A_T15WVU_A021889_20210515T181914,15WVU,2021-05-15 18:24:14.976000+00:00,942897613,27.193338,gs://gcp-public-data-sentinel-2/L2/tiles/15/W/...,0,0
4,L2A_T54UXV_A018704_20190121T013955,54UXV,2019-01-21 01:43:27+00:00,879335795,98.612651,gs://gcp-public-data-sentinel-2/L2/tiles/54/U/...,0,0


In [27]:
from sqlalchemy import create_engine
import os
USER = os.environ.get('DBUSERNAME')
PASSWORD = os.environ.get('DBPASSWORD')
ENDPOINT = os.environ.get('DBDEV')
SCHEMA = 'dev'
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{ENDPOINT}/{SCHEMA}")

In [28]:
mgrs = dfmgrs.MGRS.unique().tolist()
table = 'detect_l2a'

for loc in tqdm(mgrs):

    engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{ENDPOINT}/{SCHEMA}")
    dfup = df[df['MGRS_TILE'] == loc]
    dfup.to_sql(table, con = engine, if_exists = 'append')


  0%|          | 0/406 [00:00<?, ?it/s]